In [ ]:
from pynq import Overlay, allocate
import pynq.lib.dma
import numpy as np

DIM=16

DATA_TYPE = np.float

in_buffer_A = allocate(shape=(DIM,), dtype=DATA_TYPE, cacheable=False)
out_buffer_C = allocate(shape=(DIM,), dtype=DATA_TYPE, cacheable=False)
A = np.array([x for x in range(DIM)], dtype=DATA_TYPE)
C = np.zeros((DIM,), dtype=DATA_TYPE)

ol = Overlay('./design_1.bit')
ol.download()
    
dma0 = ol.axi_dma_0

np.copyto(in_buffer_A, A)
np.copyto(out_buffer_C, C)

def silicon():
    dma0.sendchannel.transfer(in_buffer_A)
    dma0.recvchannel.transfer(out_buffer_C)
    dma0.sendchannel.wait()
    dma0.recvchannel.wait()
    return out_buffer_C

def native():
    return np.exp(A)

In [ ]:
print("software execution:")
%time golden = native()

In [ ]:
print("hardware execution:")
%time FPGA = silicon()

In [ ]:
golden, silicon